In [72]:
import psycopg2 as sql
import pandas as pd

In [73]:
# Connect to your postgres DB
conn = sql.connect("dbname=learn user=roy")

# Open a cursor to perform database operations
cur = conn.cursor()

# Execute a query
# cur.execute("\l")
cur.execute("SELECT * FROM web_events")

# Retrieve query results
data = cur.fetchall()

In [80]:
cur.execute("create table couts( PersonID1 int, PersonID2 int, PersonID3 int ,PersonID4 int)")


In [ ]:
cl=[]
for i in cur.description:
    cl.append(i[0])

df=pd.DataFrame(data=data, columns=cl)

In [76]:
df

,id,occurred_at,channel
account_id,,,
1001,39,39,39
1011,3,3,3
1021,34,34,34
1031,2,2,2
1041,8,8,8
...,...,...,...
4461,46,46,46
4471,3,3,3
4481,5,5,5


In [ ]:
df.sort_values(by="occurred_at").count()

id             9073
account_id     9073
occurred_at    9073
channel        9073
dtype: int64

In [ ]:
df=df.groupby("account_id").count()

In [77]:
from io import StringIO

In [81]:
def copy_from_stringio(conn, df, table):
    """
    Here we are going save the dataframe in memory 
    and use copy_from() to copy it to the table
    """
    # save dataframe to an in memory buffer
    buffer = StringIO()
    df.to_csv(buffer, index_label='id', header=False)
    buffer.seek(0)
    
    cursor = conn.cursor()
    try:
        cursor.copy_from(buffer, table, sep=",")
        conn.commit()
    except (Exception, sql.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("copy_from_stringio() done")
    cursor.close()

In [82]:
copy_from_stringio(conn, df, "couts")

copy_from_stringio() done
